<a href="https://colab.research.google.com/github/sanju4n3ro/inteligencia-artificial/blob/main/predecirspamia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Paso 1: Instalar TensorFlow.js
!pip install tensorflowjs

# Paso 2: Instalar TensorFlow
!pip install tensorflow==2.16.0rc0

# Paso 3: Configurar el entorno
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
os.makedirs("spam_model", exist_ok=True)  # Crear directorio para el modelo

# Paso 4: Importar bibliotecas
import tensorflow as tf
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Paso 5: Preparar los datos
asunto = np.array([
    "¡Gana dinero rápido desde casa!",
    "Ofertas exclusivas: ¡Haz clic aquí para ganar!",
    "Felicidades, has ganado un iPhone nuevo!",
    "Préstamos aprobados en minutos, sin verificación de crédito",
    "Actualización de tu cuenta: nuevos términos y condiciones",
    "Invitación a nuestro próximo webinar sobre marketing digital",
    "Tu factura de octubre está lista para ser descargada",
])

tipo_correo = np.array([1, 1, 1, 1, 0, 0, 0], dtype=int)

# Preprocesamiento
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(asunto).toarray()

# Paso 6: Crear el modelo
modelo = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=[X.shape[1]]),
    tf.keras.layers.Dense(units=8, activation='relu'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Paso 7: Compilar el modelo
modelo.compile(
    optimizer=tf.keras.optimizers.Adam(0.1),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Paso 8: Entrenar el modelo
print("Comenzando entrenamiento...")
historial = modelo.fit(X, tipo_correo, epochs=1000, verbose=False)
print("Modelo entrenado!")

# Paso 9: Guardar el modelo
modelo.save('spam_model/tipo_de_correo.h5')

# Paso 10: Convertir el modelo a TensorFlow.js
!tensorflowjs_converter --input_format keras spam_model/tipo_de_correo.h5 spam_model

print("Modelo convertido a TensorFlow.js!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 27.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 69.0 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
    

/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2024-10-25 14:53:03.611781: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Modelo convertido a TensorFlow.js!


In [2]:
import json

# Guardar el vocabulario
vocabulario = vectorizer.get_feature_names_out()
with open('spam_model/vocabulario.json', 'w') as f:
    json.dump(vocabulario.tolist(), f)


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Clasificador de Correo Spam</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/js/bootstrap.bundle.min.js" integrity="sha384-MrcW6ZMFYlzcLA8Nl+NtUVF0sA7MsXsP1UyJoMp4YLEuNSfAP+JcXn/tWtIaxVXM" crossorigin="anonymous"></script>
    <script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@2.0.0/dist/tf.min.js"></script>

    <script type="text/javascript">
        var modelo = null;
        var vocabulario = [];

        // Cargar el modelo
        (async () => {
            console.log("Cargando modelo...");
            modelo = await tf.loadLayersModel("spam_model/model.json");
            console.log("Modelo cargado...");

            // Cargar el vocabulario
            const response = await fetch("spam_model/vocabulario.json");
            vocabulario = await response.json();
            console.log("Vocabulario cargado...");
        })();

        // Función para predecir si es spam
        function predecirSpam() {
            var asunto = document.getElementById("asunto").value;

            if (modelo != null && vocabulario.length > 0 && asunto.length > 0) {
                // Convertir el asunto a una matriz de tokens
                var palabras = asunto.toLowerCase().split(" ");
                var vector = vocabulario.map(palabra => palabras.includes(palabra) ? 1 : 0);

                // Asegurarte de que el vector tiene la longitud correcta
                if (vector.length !== vocabulario.length) {
                    console.error("El vector no tiene la longitud correcta.");
                    return;
                }

                // Convertir el vector a tensor y realizar la predicción
                var tensor = tf.tensor2d([vector]);
                var prediccion = modelo.predict(tensor).dataSync();
                var resultado = prediccion[0] >= 0.5 ? "Spam" : "No es Spam";

                document.getElementById("resultado").innerHTML = "El correo es: " + resultado;
            } else {
                document.getElementById("resultado").innerHTML = "Por favor, ingrese un asunto de correo.";
            }
        }
    </script>

    <style>
        #resultado {
            font-weight: bold;
            font-size: 1.2rem;
            text-align: center;
        }
    </style>
</head>
<body>
    <main>
        <div class="px-4 py-2 my-2 text-center border-bottom">
            <h1 class="display-5 fw-bold">Clasificador de Correo Spam</h1>
            <div class="col-lg-6 mx-auto">
                <p class="lead mb-0">Ingrese el asunto del correo para predecir si es spam o no</p>
            </div>

            <div class="b-example-divider"></div>

            <div class="container mt-5">
                <div class="row">
                    <div class="col-12 col-md-6 offset-md-3">
                        <form>
                            <div class="mb-3">
                                <label for="asunto" class="form-label">Asunto del correo</label>
                                <input type="text" class="form-control" id="asunto" placeholder="Ingrese el asunto del correo">
                            </div>
                            <div class="mb-3">
                                <button type="button" class="btn btn-primary" onclick="predecirSpam()">Predecir</button>
                            </div>
                            <div class="mb-3">
                                <label class="form-label">Resultado</label>
                                <div id="resultado">
                                    Aún no se ha hecho ninguna predicción.
                                </div>
                            </div>
                        </form>
                    </div>
                </div>
            </div>

            <div class="b-example-divider"></div>

        </div>
    </main>
</body>
</html>
